# Import Required Libraries

In [1]:
import os

from requests import get
from bs4 import BeautifulSoup

import urllib

import time
from datetime import datetime

import urllib.request

import csv
import re

import time
import random

This program was originally writen to scrape all the jewelry posts from Indiana, Ohio, and Kentucky.  The search_urls are a list of all the Craigslist sites I found in those three states.  You may need to edit for your particular search.

In [2]:

craigslist_url = 'https://cincinnati.craigslist.org'

search_urls = ['https://akroncanton.craigslist.org/d/jewelry/search/jwa', 'https://ashtabula.craigslist.org/d/jewelry/search/jwa', 'https://athensohio.craigslist.org/d/jewelry/search/jwa', 'https://chillicothe.craigslist.org/d/jewelry/search/jwa', 'https://cincinnati.craigslist.org/d/jewelry/search/jwa', 'https://cleveland.craigslist.org/d/jewelry/search/jwa', 'https://columbus.craigslist.org/d/jewelry/search/jwa', 'https://dayton.craigslist.org/d/jewelry/search/jwa', 'https://limaohio.craigslist.org/d/jewelry/search/jwa', 'https://mansfield.craigslist.org/d/jewelry/search/jwa', 'https://sandusky.craigslist.org/d/jewelry/search/jwa','https://toledo.craigslist.org/d/jewelry/search/jwa', 'https://tuscarawas.craigslist.org/d/jewelry/search/jwa', 'https://youngstown.craigslist.org/d/jewelry/search/jwa', 'https://zanesville.craigslist.org/d/jewelry/search/jwa', 'https://bgky.craigslist.org/d/jewelry/search/jwa', 'https://eastky.craigslist.org/d/jewelry/search/jwa', 'https://lexington.craigslist.org/d/jewelry/search/jwa', 'https://louisville.craigslist.org/d/jewelry/search/jwa', 'https://owensboro.craigslist.org/d/jewelry/search/jwa', 'https://westky.craigslist.org/d/jewelry/search/jwa', 'https://bloomington.craigslist.org/d/jewelry/search/jwa', 'https://evansville.craigslist.org/d/jewelry/search/jwa', 'https://fortwayne.craigslist.org/d/jewelry/search/jwa', 'https://indianapolis.craigslist.org/d/jewelry/search/jwa', 'https://kokomo.craigslist.org/d/jewelry/search/jwa', 'https://tippecanoe.craigslist.org/d/jewelry/search/jwa', 'https://muncie.craigslist.org/d/jewelry/search/jwa', 'https://richmondin.craigslist.org/d/jewelry/search/jwa', 'https://southbend.craigslist.org/d/jewelry/search/jwa', 'https://terrehaute.craigslist.org/d/jewelry/search/jwa']


In [3]:
item_anchors = []

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:12.0) Gecko/20100101 Firefox/12.0'}

##### Gather all the post urls

In [12]:
for search_url in search_urls:
    response = get(search_url, headers=headers)
    time.sleep(random.randint(3,8))
    search_soup = BeautifulSoup(response.text, 'html.parser')

    response_item_anchors = search_soup.find_all("a", {"class":"result-title hdrlnk"})

    for i in response_item_anchors:
        item_anchors.append(i)

    while True:
        if search_soup.find("a",{"class":"button next"}) is not None:
            if search_soup.find("a",{"class":"button next"})['href'] != '':
                response = get(craigslist_url + search_soup.find("a",{"class":"button next"})['href'])
                search_soup = BeautifulSoup(response.text, 'html.parser')
                new_anchors = search_soup.find_all("a", {"class":"result-title hdrlnk"})
                for anchor in new_anchors:
                    item_anchors.append(anchor)
            else:
                break
        else:
            break 

##### Scrape information from each post

In [13]:
previous_links = []
records = []
for link in item_anchors:
    if link['href'] in previous_links:
        continue
    else:
        previous_links.append(link['href'])
        item_response = get(link['href'], headers=headers)
        item_soup = BeautifulSoup(item_response.text,'html.parser')
        if item_soup.find("span", {"id":"titletextonly"}) is not None:
            item_title = item_soup.find("span", {"id":"titletextonly"}).get_text()
            cleaner_title = re.sub(r'[^\x00-\x7F]+','', item_title)
            clean_title = re.sub(r'[<>:"\/\\|?*]','',cleaner_title)
            clean_title = clean_title.replace('\t',' ')
        else:
            clean_title = "no name"

        if item_soup.find("span", {"class":"price"}) is not None:
            item_price = item_soup.find("span", {"class":"price"}).get_text()
        else:
            item_price = "not listed"
        if item_soup.find("div", {"class": "slide first visible"}) is not None:
            item_image_url = item_soup.find("div", {"class": "slide first visible"}).img['src']
        else:
            item_image_url = "no image"

        if item_soup.find("time", {"class":"date timeago"}) is not None:
            item_raw_date = item_soup.find("time", {"class":"date timeago"})['datetime']
            item_parse_date = item_raw_date[:-2] + ":" + item_raw_date[-2:]
            d = datetime.fromisoformat(item_parse_date)
        else:
            d = datetime.min

        records.append([clean_title, item_price, d.strftime("%a %b %d %I:%M %p %Z") , item_image_url, link['href']])


##### Create file for information and photos to go

In [14]:
base_repository = os.path.normpath(r'C:\path\to\new\file')
file_time = datetime.now().strftime("%b %d %Y %Hh%Mm%Ss")

new_file = os.mkdir(os.path.join(base_repository, file_time ))

os.chdir(os.path.join(base_repository, file_time ))

new_repository = os.path.normpath(os.path.join(base_repository, file_time ))


##### Write post information to csv file, save to new folder

In [15]:
with open('cragislist_update.csv', mode='w', newline='\n', encoding="utf-8") as output_file:
    ghostwriter = csv.writer(output_file)
    ghostwriter.writerow(['Item Title', 'item price', 'post date', 'image url', 'Post url'])
    ghostwriter.writerows(records)

##### Save pictures to new file

In [16]:
img_count = 0
for record in records:
    if record[3] != 'no image':
        urllib.request.urlretrieve(record[3], os.path.join(new_repository,   record[0] + '_' + str(img_count) + '.png') )
        img_count += 1